# v9fs performance studies

First thing we need to do is start a qemu with a kernel.  The versions of both of these matter if what we are doing is qemu-virtio as transport (and probably also have an impact on qemu network performance)

TODO:
- [ ] TASK: add build configuraton for [qemu | kernel | u-root | etc]

# Setup Test Environment

The following command starts qemu in the background.  You can optionally set the QEMU command line and KERNEL to use as environment variables.
You must be setup for kvm and the user must have permission to start kvm instances without sudo (usually by being in group kvm).

Okay, so this all works ... but we don't have any nice way to cleanup, so we'll have to work that out at some point.

In [ ]:
QEMU = "../qemu/build/qemu-system-x86_64"
KERNEL = "../build/arch/x86_64/boot/bzImage"


In [ ]:
!sg kvm -c '{QEMU} -kernel \
    {KERNEL} \
	-cpu  max \
    -s   \
    -enable-kvm \
    -m 8192m \
    -machine q35  \
    -initrd /tmp/initramfs.linux_amd64.cpio \
    -object rng-random,filename=/dev/urandom,id=rng0 \
    -device virtio-rng-pci,rng=rng0 \
    -device vhost-vsock-pci,guest-cid=42 \
    -device virtio-net-pci,netdev=n1 \
    -netdev user,id=n1,hostfwd=tcp:127.0.0.1:17010-:17010,net=192.168.1.0/24,host=192.168.1.1 \
    -debugcon file:debug.log -global isa-debugcon.iobase=0x402 \
    -fsdev local,security_model=passthrough,id=fsdev0,path=/tmp \
    -device virtio-9p-pci,id=fs0,fsdev=fsdev0,mount_tag=hostshare \
    -daemonize'

# Run a test

Okay, so now we need some simple tests to run in this environment.  We'll do this all over cpu.  Most operations will be stored in bench.py, but we'll configure them here



In [ ]:
# run this cell during debug to make sure underlying python modules reload
%load_ext autoreload
%autoreload 2

In [ ]:
#!/usr/bin/python
from bench import BenchTarget, BenchProgress, bandwidth
from IPython.display import clear_output, display
import logging
import pandas as pd

npfs = BenchTarget("npfs", ["../npfs/fs/npfs", "-p", "5640", "-w", "8", "-s"], "/mnt/9/test", "netfstab")
p9ufs = BenchTarget("p9ufs",["../p9/cmd/p9ufs/p9ufs","0.0.0.0:5640"], "/mnt/9/test", "netfstab")
virtio = BenchTarget("virtio","","/mnt/9/test", "virtiofstab")
cpud = BenchTarget("cpu", "", "./test", "fstab")

testcfgs = [ virtio, npfs, p9ufs, cpud ]
blocksizes = [ 131072, 65536, 32768, 16384, 8192, 4096, 1024, 512 ]

def stepDesc( notes, s, t):
    clear_output(wait=True)
    print( notes + "["+str(s)+"/"+str(t)+"]")

prog = BenchProgress(len(testcfgs)*len(blocksizes)*5, stepDesc)

logger = logging.getLogger()
logging.basicConfig(filename="log.txt", level=logging.DEBUG)

results = bandwidth( testcfgs, blocksizes, 5, blocksizes[0]*5000, "notebook test", prog, logger) 

clear_output(wait=True)
display(results)


# Process results from a particular run

In [70]:
len(results['Target'].unique())

4

In [64]:

processed = pd.DataFrame( { "Run Config": [], "Target": [], "Blocksize": [],  "MeanBW": [], "MinBW": [], "MaxBW": [], "MaxStdD": [] } )
row = results.query('Target=="virtio" and Blocksize == 131072')
display(row)
processed.loc[len(processed.index)] = [ row['Run Config'][0], row['Target'][0], row['Blocksize'][0], row['Bandwidth'].mean(),row['Bandwidth'].min(),row['Bandwidth'].max(), row['Bandwidth'].std() ]

display(processed)

,Run Config,Target,Blocksize,Iteration:,Bandwidth
0,97,virtio,131072,0,16.9
1,97,virtio,131072,1,16.9
2,97,virtio,131072,2,15.9
3,97,virtio,131072,3,16.5
4,97,virtio,131072,4,15.7


,Run Config,Target,Blocksize,MeanBW,MinBW,MaxBW,MaxStdD
0,97,virtio,131072,16.38,15.7,16.9,0.55857


In [87]:
cs = results['Target'].unique()
bs = results['Blocksize'].unique()
p2 = pd.DataFrame( { "Target": [], "Blocksize": [],  "MeanBW": [], "MinBW": [], "MaxBW": [], "MaxStdD": [] } )
for c in cs:
    for b in bs:
        q = 'Target == "'+c+'" and Blocksize == '+str(b)
        row = results.query(q)
        first = row.head(1)
        p2.loc[len(p2.index)] = [ c, b, row['Bandwidth'].mean(),row['Bandwidth'].min(),row['Bandwidth'].max(), row['Bandwidth'].std() ]
display(p2)

,Target,Blocksize,MeanBW,MinBW,MaxBW,MaxStdD
0,virtio,131072,16.38,15.7,16.9,0.558570
1,virtio,65536,16.84,16.5,17.5,0.409878
2,virtio,32768,16.40,15.7,17.1,0.556776
3,virtio,16384,16.50,15.7,17.0,0.543139
4,virtio,8192,15.78,15.0,16.5,0.549545
5,virtio,4096,14.84,14.1,15.7,0.676757
6,virtio,1024,10.34,10.1,10.7,0.260768
7,virtio,512,7.14,7.0,7.3,0.134164
8,npfs,131072,10.72,6.4,16.6,5.115858
9,npfs,65536,15.86,15.1,16.3,0.502991


In [96]:
import bench

processed = bench.refine(results)
display(processed)

,Target,Blocksize,MeanBW,MinBW,MaxBW,MaxStdD
0,virtio,131072,16.38,15.7,16.9,0.558570
1,virtio,65536,16.84,16.5,17.5,0.409878
2,virtio,32768,16.40,15.7,17.1,0.556776
3,virtio,16384,16.50,15.7,17.0,0.543139
4,virtio,8192,15.78,15.0,16.5,0.549545
5,virtio,4096,14.84,14.1,15.7,0.676757
6,virtio,1024,10.34,10.1,10.7,0.260768
7,virtio,512,7.14,7.0,7.3,0.134164
8,npfs,131072,10.72,6.4,16.6,5.115858
9,npfs,65536,15.86,15.1,16.3,0.502991
